In [224]:
import numpy as np
from scipy.optimize import linprog

In [225]:

def pretty_print_weekly(x, items):
  M = []

  ctr = 0
  row = []
  for i in range(1, len(x)+1):
    row.append(x[i-1])
    if (i % 7 == 0):
      row.insert(0, items[ctr])
      M.append(row)
      ctr += 1
      row = []

  day = [[' ', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']]
  print('\n'.join([''.join(['{:8}'.format(ele) for ele in row]) for row in day]))
  print('\n'.join([''.join(['{:8}'.format(ele) for ele in row]) for row in M]))


In [226]:

def build_c(prices):
  c = []
  for p in prices:
    for i in range(0, 7):
      c += [p]
  return c


In [227]:

def constrain_weekly_min(A, b, idx_item, min_amount, n_items):
  A.append([-1 for i in range(0, 7*n_items)])
  b += [-min_amount]
  return (A, b)

def constrain_weekly_max(A, b, idx_item, max_amount, n_items):
  A.append([1 for i in range(0, 7*n_items)])
  b += [max_amount]
  return (A, b)

def constrain_weekly_max_item(A, b, idx_item, max_amount, n_items):
  a = [0]*(7*n_items)
  for i in range(0, 7):
    a[(7*idx_item)+i] = 1
  A.append(a)
  b += [max_amount]
  return (A, b)

def constrain_daily_max_item(A, b, idx_item, idx_day, max_amount, n_items):
  a = [0]*(7*n_items)
  a[(7*idx_item)+idx_day] = 1
  A.append(a)
  b += [max_amount]
  return (A, b)


In [228]:

def find_min_cost_local(c, A_eq, b_eq, bounds, items, verbose=False):
  res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
  if res.fun is not None and verbose:
    print(res.message)
    print('raw printing..')
    print(res.x)
    print('\n'+"pretty printing..")
    pretty_print_weekly(res.x, items)
  return (res.fun)

def find_min_cost_global(items, prices_markets, prices, path, it):
  if it == len(items):
    n = len(prices)
    c = build_c(prices)

    A_eq = [[0]*(7*n)]
    b_eq = [0]

    A_eq, b_eq = constrain_weekly_max_item(A_eq_run, b_eq_run, 1, 50, n)
    A_eq, b_eq = constrain_weekly_max_item(A_eq_run, b_eq_run, 3, 25, n)
    A_eq, b_eq = constrain_weekly_max_item(A_eq_run, b_eq_run, 5, 250, n)
    A_eq, b_eq = constrain_daily_max_item(A_eq_run, b_eq_run, 3, 0, 10, n)
    A_eq, b_eq = constrain_daily_max_item(A_eq_run, b_eq_run, 3, 1, 3, n)
    A_eq, b_eq = constrain_daily_max_item(A_eq_run, b_eq_run, 5, 2, 100, n)
    A_eq, b_eq = constrain_daily_max_item(A_eq_run, b_eq_run, 2, 5, 222, n)

    bounds = [(1, None)]*(7*n)

    price = find_min_cost_local(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, items=items)
    print(prices, " - ", path, " : ", price)
    return (price, prices, path)
  else:
    global_min = 1000000000
    prices_min = []
    path_min = []
    for i, prices_market in enumerate(prices_markets):
      local_min, prices_min_, path_min_ = find_min_cost_global(items, prices_markets, prices+[prices_market[it]], path+[i], it+1)
      if local_min < global_min:
        global_min = local_min
        prices_min = prices_min_
        path_min = path_min_
    return (global_min, prices_min, path_min)

**Beggining of execution**

In [229]:

items_run = ['Appel', 'Garlic', 'Onion', 'Potato', 'Rice', 'Tomato']

prices_market1 = [
            2000, # Appel
            1800, # Garlic
            6000, # Onion
            3000, # Potato
            3500, # Rice
            1500, # Tomato
          ]

prices_market2 = [
            1800, # Appel
            1800, # Garlic
            6200, # Onion
            3200, # Potato
            3400, # Rice
            1900, # Tomato
          ]


**Test suite**

In [230]:
n_run = len(prices_market1)

c_run = build_c(prices_market1)


# Restrictions only <=


# Restrictions only =
A_eq_run = [[0]*(7*n_run)]
b_eq_run = [0]

A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 1, 50, n_run)
A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 3, 25, n_run)
A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 5, 250, n_run)
A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 3, 0, 10, n_run)
A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 3, 1, 3, n_run)
A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 5, 2, 100, n_run)
A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 2, 5, 222, n_run)

print(c_run)
print(A_eq_run, b_eq_run)


bounds_run = [(1, None)]*(7*n_run)

[2000, 2000, 2000, 2000, 2000, 2000, 2000, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3500, 3500, 3500, 3500, 3500, 3500, 3500, 1500, 1500, 1500, 1500, 1500, 1500, 1500]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [231]:
price = find_min_cost_local(c_run, A_eq=A_eq_run, b_eq=b_eq_run, bounds=bounds_run, items=items_run, verbose=True)
print(price)

Optimization terminated successfully. (HiGHS Status 7: Optimal)
raw printing..
[  1.   1.   1.   1.   1.   1.   1.  44.   1.   1.   1.   1.   1.   1.
   1.   1.   1.   1.   1. 222.   1.  10.   3.   8.   1.   1.   1.   1.
   1.   1.   1.   1.   1.   1.   1. 145.   1. 100.   1.   1.   1.   1.]

pretty printing..
        Mon     Tue     Wed     Thu     Fri     Sat     Sun     
Appel        1.0     1.0     1.0     1.0     1.0     1.0     1.0
Garlic      44.0     1.0     1.0     1.0     1.0     1.0     1.0
Onion        1.0     1.0     1.0     1.0     1.0   222.0     1.0
Potato      10.0     3.0     8.0     1.0     1.0     1.0     1.0
Rice         1.0     1.0     1.0     1.0     1.0     1.0     1.0
Tomato     145.0     1.0   100.0     1.0     1.0     1.0     1.0
1946500.0


In [232]:
find_min_cost_global(items_run, [prices_market1, prices_market2], [], [], 0)

[2000, 1800, 6000, 3000, 3500, 1500]  -  [0, 0, 0, 0, 0, 0]  :  1946500.0
[2000, 1800, 6000, 3000, 3500, 1900]  -  [0, 0, 0, 0, 0, 1]  :  2046500.0
[2000, 1800, 6000, 3000, 3400, 1500]  -  [0, 0, 0, 0, 1, 0]  :  1945800.0
[2000, 1800, 6000, 3000, 3400, 1900]  -  [0, 0, 0, 0, 1, 1]  :  2045800.0
[2000, 1800, 6000, 3200, 3500, 1500]  -  [0, 0, 0, 1, 0, 0]  :  1951500.0
[2000, 1800, 6000, 3200, 3500, 1900]  -  [0, 0, 0, 1, 0, 1]  :  2051500.0
[2000, 1800, 6000, 3200, 3400, 1500]  -  [0, 0, 0, 1, 1, 0]  :  1950800.0
[2000, 1800, 6000, 3200, 3400, 1900]  -  [0, 0, 0, 1, 1, 1]  :  2050800.0
[2000, 1800, 6200, 3000, 3500, 1500]  -  [0, 0, 1, 0, 0, 0]  :  1992100.0
[2000, 1800, 6200, 3000, 3500, 1900]  -  [0, 0, 1, 0, 0, 1]  :  2092100.0
[2000, 1800, 6200, 3000, 3400, 1500]  -  [0, 0, 1, 0, 1, 0]  :  1991400.0
[2000, 1800, 6200, 3000, 3400, 1900]  -  [0, 0, 1, 0, 1, 1]  :  2091400.0
[2000, 1800, 6200, 3200, 3500, 1500]  -  [0, 0, 1, 1, 0, 0]  :  1997100.0
[2000, 1800, 6200, 3200, 3500, 1900]  

(1944400.0, [1800, 1800, 6000, 3000, 3400, 1500], [1, 0, 0, 0, 1, 0])